# Sprachmodelle I - Text und Token

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline
pd.options.display.float_format = "{:,.4f}".format

Wir ladene einen ersten Text aus einer Datei und geben ihn aus:

In [ ]:
with open('txt/zauberlehrling.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    
print(text)

## Von Text zu Zahlen und wieder zurück

Um Texte im Computer verarbeiten zu können, müssen wir sie irgendwie auf Zahlen abbilden. Dazu brauchen wir irgendeine Vorschrift. Eine einfache Vorschrift, die auch traditionell in Computern verwendet wird, ist es jedem Buchstaben und jedem Zeichen eine Zahl zuzuordnen. Bei der Entwicklung der Sprachmodelle stellte sich jedoch heraus, dass das zusammenfassen von einzelnen Buchstaben und Zeichen zu besseren Ergebnissen führt. Diese zusammengefassten Zeichen nennt man Token. Der Einfachheit halber haben wir die Buchstaben eines Wortes immer als einen Token aufgefasst und jedes Sonderzeichen bzw. jeden Zeilenumbruch als einen eigenen Token.

In [ ]:
from fws.tokenizer import FWSTokenizer
tokenizer = FWSTokenizer(text)

Zunächst machen wir den Computer bekannt mit den allen Texten, die er beherrschen soll. Dazu erzeugen wir einen Tokenizer und füttern ihn erstmal nur mit dem Gedicht 'Der Zauberlehrling'. Der Tokenizer zerlegt das Gedicht in Token und merkt sich, welche Token vorkamen. Die ersten 15 sind:

In [ ]:
df = pd.DataFrame(
    data={'token':tokenizer.vocab_str(), 'output':tokenizer.decode_list(tokenizer.vocab_int())}, 
    index=tokenizer.vocab_int())
df.head(15)

Insgesamt hat der Tokenizer so viele unterschiedliche Token gefunden:

In [ ]:
print(tokenizer.vsize)

Der Tokenizer kann aus Texten Zahlen machen und aus Zahlen wieder Texte. Aber er kann nur mit den Texten umgehen, die er auch kennengelernt hat:

In [ ]:
neuer_text = "Ich checke das nicht"
neue_idx_seq = tokenizer.encode(tokenizer.tokenize(neuer_text))
print(neue_idx_seq)

Wenn wir die Zahlen wieder in Buchstaben verwandeln fällt es auf:

In [ ]:
print(tokenizer.decode(neue_idx_seq))

#### Aufgabe 1:

##### 0,5+0,5 = 1 Punkt

- F: Benennen Sie einen Token im Text 'Der Zauberlehrling', der mehr als einmal vorkommt
- A:
- F: Warum kommt beim zurückverwandeln von 'Ich checke das nicht' nur noch 'das nicht' raus?
- A:

Zu wissen welche Token es gibt, ist auch hilfreich um aus den bekannten Token neue Texte erzeugen:

## Auf der Suche nach einem Modell, das Texte schreibt

### Versuch 1

Wir werfen alle Token in einen Sack, schütteln kräftig und ziehen zufällig 32 Stück hinterinander heraus:

In [ ]:
rng = np.random.default_rng()
gen_idx_seq = rng.integers(0, tokenizer.vsize - 1, size=32)
print(tokenizer.decode(gen_idx_seq))

### Vorbereitung Versuch 2

Für den zweiten Versuch zählen wir, wievele Token im Gedicht insgesamt vorkamen und wie häufig jeder einzelne vorkaum. Daraus können wir die relative Häufigkeit berechnen:

In [ ]:
toks = tokenizer.tokenize(text)
print(len(toks))

In [ ]:
c = torch.zeros(tokenizer.vsize, dtype=torch.int32)
for tok in toks:
    c[tokenizer.idx(tok)] += 1

df['count'] = c 
df['frq'] = df['count'] / len(toks)

df.head(20)

In [ ]:
df_sorted = df.sort_values('frq', ascending=False)
df_sorted.head(20)

Die relative Häufigkeit summiert sich auf 1:

In [ ]:
df['frq'].sum()

### Versuch 2

Jetzt werfen wir wieder alle Token die wir kennen in einen Sack, schütteln Kräftig und ziehen mit einer magischen Hand jetzt wieder zufällig, aber berücksichtigen dabei die relative Häufigkeit:

In [ ]:
gen_idx_seq = rng.choice(tokenizer.vocab_int(), size=32, replace=True, p=df['frq'].to_numpy())
print(tokenizer.decode(gen_idx_seq))

#### Aufgabe 2

##### 1+1+1 = 3 Punkte

Führen Sie die Versuche 1 und 2 mehrfach aus und Vergleichen Sie die Ergebnisse.

- F: Was sind die größten Unterschiede zwischen den Ergebnissen?
- A:
- F: Welcher Versuch produziert Ergebnisse, die näher an einem Gedicht sind?
- A:
- F: Warum ergibt keiner der Texte Sinn?
- A:

### Vorbereitung Versuch 3

Für den dritten Versuch zählen wir jetzt nicht nur, wie oft ein Token vorkommt, sondern auch, welcher Token wie oft davor steht. Die Zahlen sammeln wir in einer Matrix. Das ist eine Tabelle, in der jede Zeile und jede Spalte einem Token entspricht, und jede Zelle wie oft die beiden Token aus Zeile und Spalte im Text hintereinander vorkommen.

In [ ]:
N = torch.zeros((tokenizer.vsize, tokenizer.vsize), dtype=torch.int32)
toks = tokenizer.tokenize(text)
for t1, t2 in zip(toks, toks[1:]):
    ix1 = tokenizer.idx(t1)
    ix2 = tokenizer.idx(t2)
    N[ix1, ix2] += 1
N

Damit wir die Matrix besser lesen können, stellen wir sie nochmal mit Überschriften für die Zeilen und Spalten dar:

In [ ]:
from IPython.display import display, HTML
bidf = pd.DataFrame(N.numpy(), columns=tokenizer.vocab_str(), index=tokenizer.vocab_str())
pd.options.display.float_format = '{:,.2f}'.format
bidf.head(10)

Weil die Tabelle recht groß ist, ist sie hier nicht vollständig dargestellt. Wir können aber herausfinden welchen Wert eine Zelle in der Tabelle hat. So können wir herausfinden, wie oft die Token 'ich' und 'bin' hintereinander im Text vorkommen.

In [ ]:
tok = 'ich'
next_tok = 'bin'
N[tokenizer.idx(tok), tokenizer.idx(next_tok)].item()

Wir können auch herausfinden, wie oft ein bestimmter Token im Text auftaucht, indem wir die Zeile (oder Spalte) dieses Tokens aufsummieren.

In [ ]:
tok = 'ich'
N[tokenizer.idx(tok), :].sum().item()

Wir können auch eine ganze Tabellenzeile ausgeben, beispielsweise um zu sehen, welche anderen Token wie oft auf 'ich' folgen:

In [ ]:
tok = 'ich'
N[tokenizer.idx(tok), :]

Da das schwer zu lesen ist, können wir uns mit ein bisschen mehr code auch die Liste aller Token ausgeben lassen, die auf 'ich' folgen.

In [ ]:
tok = 'ich'
ich_next = [tokenizer.tok(idx) for idx,cnt in enumerate(N[tokenizer.idx(tok), :].tolist()) if cnt > 0]
print(ich_next)

Erinnern wir uns an Versuch 2, wo wir die relative Häufigkeit der einzelnen Token berechnet haben. Mit Hilfe der Matrix die wir oben erzeugt haben, können wir jetzt die relative Häufigkeit für jeden Token angeben, der auf einen anderen folgt:

#### Aufgabe 3

##### 0,5+0,5+0,5+0,5 = 2 Punkte

In jeder Zeile der Tabelle oben lässt sich ablesen, welcher Token wie oft auf den Token am Beginn dieser Zeile folgt. 

Beispiel: In der 8. Zeile steht ganz links 'Ach' und es lässt sich ablesen, dass in dieser Zeile eine 4 in der Spalte \<CM\> und eine 2 in der Spalte \<EM\> steht. Das Bedeutet im Text folgt auf 'Ach' viermal ein Komma und zweimal Ausrufezeichen.
- F: Wie viele mögliche Token folgen auf 'und'?
- A:
- F: Gibt es einen Token der zweimal hintereinander vorkommt? Wo in der Tabelle können Sie das ablesen? 
- A:
- F: Wo kann man ablesen, welcher Token vor einem Zeilenumbruch (Token: \<NL\>) stehen?
- A:
- F: Wie kann man ablesen, wie oft ein bestimmter Token insgesamt vorkommt?
- A:

### ... auf der Zielgeraden

In [ ]:
P = N.float()
P /= P.sum(1, keepdim=True)
P

Wir können diese Matrix der relativen Häufigkeiten auch als Bild darstellen:

In [ ]:
plt.imshow(P)

### Versuch 3

Im dritten Versuch gehen wir schrittweise vor. Wir starten mit einem Token, mit dem Üblicherweise ein Gedicht anfängt, nämlich einer neuen Zeile. Jetzt werfen wir alle Token, die auf die neue Zeile folgen in einen Sack und schütteln kräftig. Dann ziehen mit einer magischen Hand jetzt wieder zufällig, aber berücksichtigen dabei die relative Häufigkeit der Token, die unserem Startoken folgen. Wir haben jetzt einen neuen Token und werfen wieder alle Token, die auf diesen neuen Token folgen in einen Sack... 

In [ ]:
start_token = '<NL>'
max_tokens = 32
gen_idx_seq = [tokenizer.idx(start_token)]
while True:
    probs = P[gen_idx_seq[-1]]
    idx = torch.multinomial(probs, num_samples=1).item()
    gen_idx_seq.append(idx)
    if len(gen_idx_seq) >= max_tokens:
        break

print(tokenizer.decode(gen_idx_seq))

#### Aufgabe 4

##### 1+1+2 = 4 Punkte

- F: Was sieht man auf dem Bild oben?
- A:
- F: Wiederholen Sie den dritten Versuch ein paar mal. Was beobachten sie im Vergleich zu, zweiten Versuch?
- A:
- F: In Versuch 3 haben wir unser erstes Sprachmodell erstellt. Wie würden Sie es bezeichnen und was sind die Parameter des Modells? 
- A:

## Wie messen wir wie gut unser Modell ist?

Wie groß ist die durchschnittliche realtive Häufigekit ("Wahrscheinlichkeit") eines Tokens?

In [ ]:
1/tokenizer.vsize

In [ ]:
def berechne_fehler(tokenizer, text, pMatrix):
    data = []
    toks = tokenizer.tokenize(text)
    likelihood = 1
    log_likelihood = 0.0
    print_pairs = 10
    count = 0
    for t1, t2 in zip(toks, toks[1:]):
        ix1 = tokenizer.idx(t1)
        ix2 = tokenizer.idx(t2)
        p = pMatrix[ix1, ix2]
        logp = torch.log(p)
        likelihood *= p
        log_likelihood += logp
        count += 1
        data.append({'Token 1':t1, 'Token 2':t2, 'Likelihood':p.item(), 'Log Likelihood':logp.item(), 'Negative Log Likelihood':-logp.item()})
        
    return pd.DataFrame(data)

In [ ]:
edf = berechne_fehler(tokenizer, text, P)
edf.head(10)

In [ ]:
print(f"Likelihood: {edf['Likelihood'].product()}") 
print(f"Log Likelihood: {edf['Log Likelihood'].sum()}")
log_likelihood = edf['Likelihood'].product()
neg_log_likelihood = -log_likelihood
print(f"Negative Log Likelihood: {edf['Negative Log Likelihood'].sum()}")
neg_log_likelihood = -log_likelihood
print(f"Average Negative Log Likelihood (Fehler): {edf['Negative Log Likelihood'].mean()}")

In [ ]:
edf[edf['Likelihood'] == 0].head()

In [ ]:
edf = berechne_fehler(tokenizer, "ich checke das nicht", P)
edf.head(10)

In [ ]:
P2 = (N + 1).float()
P2 /= P2.sum(1, keepdim=True)

In [ ]:
edf = berechne_fehler(tokenizer, "ich checke das nicht", P2)
edf.head(10)